In [ ]:
import sepal_ui.sepalwidgets as sw

from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.tile.map_tile import MapTile
from component.tile.questionnaire_tile import QuestionnaireTile
from component.tile.recipe_tile import RecipeTile

from component.message import cm

In [ ]:
aoi_tile = AoiTile()
questionnaire_tile = QuestionnaireTile()
map_tile = MapTile()
dashboard_tile = DashboardTile()
recipe_tile = RecipeTile(aoi_tile, questionnaire_tile, map_tile, dashboard_tile)

In [ ]:
# Create an appBar
app_bar = sw.AppBar(cm.app.title)

In [ ]:
%run 'about_ui.ipynb'

In [ ]:
app_content = [
    aoi_tile,
    questionnaire_tile,
    map_tile,
    dashboard_tile,
    recipe_tile,
]

In [ ]:
items = [
    sw.DrawerItem(cm.app.drawer.about, "mdi-help-circle", card="about_tile"),
    sw.DrawerItem(cm.app.drawer.recipe, "mdi-map-marker-check", card="recipe_tile"),
    sw.DrawerItem(cm.app.drawer.aoi, "mdi-map-marker-check", card="aoi_tile"),
    sw.DrawerItem(
        cm.app.drawer.question, "mdi-file-question", card="questionnaire_tile"
    ),
    sw.DrawerItem(cm.app.drawer.map, "mdi-map", card="map_tile"),
    sw.DrawerItem(cm.app.drawer.dashboard, "mdi-view-dashboard", card="dashboard_tile"),
]

code_link = "https://github.com/sepal-contrib/se.plan"
wiki_link = "https://docs.sepal.io/en/latest/modules/dwn/seplan.html"
issue_link = "https://github.com/sepal-contrib/se.plan/issues/new"

app_drawer = sw.NavDrawer(items, code=code_link, wiki=wiki_link, issue=issue_link)

In [ ]:
# build the Html final app by gathering everything
app = sw.App(tiles=app_content, appBar=app_bar, navDrawer=app_drawer).show_tile(
    "about_tile"
)

In [ ]:
app